<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/gender_age_estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2, glob, dlib
from google.colab.patches import cv2_imshow

# age_list: 기존에 모델을 개발한 사람이 만들어 놓은 것
age_list = ['(0, 2)','(4, 6)','(8, 12)','(15, 20)','(25, 32)','(38, 43)','(48, 53)','(60, 100)']
# gender_list: 기존에 모델을 개발한 사람이 만들어 놓은 것
gender_list = ['Male', 'Female']

# 얼굴을 detecting 하기위한 face detector
detector = dlib.get_frontal_face_detector()

# 기존에 학습되어있던 데이터로 진행
# CNN기반 나이 판별 학습데이터
age_net = cv2.dnn.readNetFromCaffe(
          '/content/drive/My Drive/insuretech/gender/models/deploy_age.prototxt', 
          '/content/drive/My Drive/insuretech/gender/models/age_net.caffemodel')

# CNN기반 성별 판별 학습데이터
gender_net = cv2.dnn.readNetFromCaffe(
          '/content/drive/My Drive/insuretech/gender/models/deploy_gender.prototxt', 
          '/content/drive/My Drive/insuretech/gender/models/gender_net.caffemodel')

# 테스트 결과
# 보정된 사진에는 이상한 결과가 나옴

# glob을 통해서 이미지 파일을 전부 가져옴
img_list = glob.glob('/content/drive/My Drive/insuretech/gender/img/*')

# 이미지를 하나씩 읽음
for img_path in img_list:
  # image 읽기
  img = cv2.imread(img_path)

  # dlib의 detector객체 선언 후 이미지 대입
  faces = detector(img)

  # 찾은 얼굴 갯수만큼 이터레이팅
  for face in faces:
    # 얼굴의 상하좌우 좌표
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()

    # 얼굴만 추출
    # 행 데이터 -> y
    # 열 데이터 -> x
    face_img = img[y1:y2, x1:x2].copy()

    # 계산을 하기 위해서
    # 블롭이란 영상 등의 데이터를 포함할 수 있는 다차원 데이터 표현 방식
    # OpenCV에서 블롭은 Mat 타입의 4차원 행렬로 표현됩니다.
    # mean은 각 BGR 에서 빼야하는 값, 모든 데이터에서 추출한 각 채널의 평균 -> 노이즈 제거
    blob = cv2.dnn.blobFromImage(face_img, scalefactor=1, size=(227, 227),
      mean=(78.4263377603, 87.7689143744, 114.895847746),
      swapRB=False, crop=False)

    # predict gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]

    # predict age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]

    # visualize
    cv2.rectangle(img, (x1, y1), (x2, y2), (255,255,255), 2)
    overlay_text = '%s %s' % (gender, age)
    cv2.putText(img, overlay_text, org=(x1, y1), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
      fontScale=1, color=(0,0,0), thickness=10)
    cv2.putText(img, overlay_text, org=(x1, y1),
      fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255), thickness=2)

  cv2_imshow(img)
  cv2.imwrite('/content/drive/My Drive/insuretech/gender/result/%s' % img_path.split('/')[-1], img)